In [2]:
#!pip install datetime
#!pip install numpy
#!pip install pandas
#!pip install xgboost
#!pip install scikit-learn
#!pip install agentds-bench
#!pip install imblearn

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [imblearn]d-learn]


In [23]:
import pandas as pd
import numpy as np
import xgboost
import sklearn
from itertools import product
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score, make_scorer
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from imblearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [9]:
# Selected features for our model
labels = ['Online', 'Channel', 'MCC_Group', 'Amount',
              'Type', 'Balance', 'Limit', 'Type_dst', 'Balance_dst',
              'Limit_dst', 'Age', 'Tenure', 'CreditScore', 'AnnualSalary',
              'Device_Count', 'Action_Count', 'Login_Count', 'Logout_Count',
              'Account_View_Count', 'Payment_Count', 'Transfer_Count',
              'Day_of_Week', 'Timestamp', 'in_home_city', 'prev_trans_in_city',
              'times_device_used', 'unique_devices_past', 'days_since_open',
              'days_since_open_dst', 'time_since_last_txn_hours',
              'xacts_earlier_today', 'avg_amount_past', 'amount_ratio',
              'amount_dev', 'mcc_group_prev_count']

#Check Class Imbalance of FraudLabel
df = pd.read_csv('final_train.csv')
X_df = df[labels]
Y_df = df[['FraudLabel']]
print(Counter(df['FraudLabel']))

Counter({0: 151808, 1: 507})


In [15]:
X = X_df.to_numpy()
y = Y_df.to_numpy()
print(y)
print(X[0:2, :])

[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]
[[0 1 1 44.41617907 0 300.342310530943 nan -1 nan nan 53.84044406467309
  7.325337186630032 781.4387832676174 49665.18770258372 2 0 0 0 0 0 0 0 9
  False nan nan nan 56 nan 0 0 nan nan nan 0]
 [0 1 2 68.53060371 0 300.342310530943 nan -1 nan nan 53.84044406467309
  7.325337186630032 781.4387832676174 49665.18770258372 2 0 0 0 0 0 0 2 3
  False nan nan nan 57 nan 24 0 44.41617907 1.5429198356300664
  0.5429198356300664 0]]


In [16]:

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, shuffle=True)


In [34]:
# Train the model
# neg_count = np.sum(y_train == 0)
# pos_count = np.sum(y_train == 1)
# scale_pos_weight_value = np.sqrt(neg_count / pos_count) # square root can be used if scale_pos_weight is too high
#Pipeline = OverSample Minority then Undersample Majority
imputer = SimpleImputer(strategy='median')
over = SMOTE(random_state=42, sampling_strategy=0.1)
under  = RandomUnderSampler(sampling_strategy=0.5)
steps = [('imputer', imputer), ('o',over), ('u',under)]
pipeline = Pipeline(steps=steps)
X_train_res, Y_train_res = pipeline.fit_resample(X_train, Y_train)

print(Counter(Y_train_res))

Counter({np.int64(0): 24290, np.int64(1): 12145})


In [ ]:
# xgb = XGBClassifier(objective='binary:logistic', eval_metric='aucpr', random_state=42, scale_pos_weight = scale_pos_weight_value, n_estimators=1000)
xgb = XGBClassifier(
    objective='binary:logistic',
    eval_metric='aucpr',
    random_state=42,
    n_estimators=1000
)

# Cross validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
macro_f1_scorer = make_scorer(f1_score, average='macro')
param_grid = {
    'max_depth': [4, 5],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'eta': [0.01, 0.05, 0.1],
}
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    cv=cv,
    scoring=macro_f1_scorer,
    n_jobs=-1,
    verbose=1
)
grid_search.fit(X_train, Y_train)
print(f"Best parameters found: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_:.4f}")
best_xgb_model = grid_search.best_estimator_

preds = best_xgb_model.predict(X_test)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


In [ ]:
best_params = grid_search.best_params_
final_xgb = XGBClassifier(
    **best_params,
    early_stopping_rounds=20,
    n_estimators=1000,
    random_state=42,
    eval_metric='aucpr'
)


final_xgb.fit(
    X_train_res,
    Y_train_res,
    eval_set=[(X_test, y_test)],
    verbose=False
)
best_xgb_model.save_model('xgboost_model.json')
#best_xgb_model = XGBClassifier()
#best_xgb_model.load_model('xgboost_model.json')
#print(best_xgb_model.get_params())

In [ ]:
y_test = y_test.reshape(-1)
preds = best_xgb_model.predict(X_test)
F1 = f1_score(y_test, preds, average='macro')
precision = precision_score(y_test, preds)
recall = recall_score(y_test, preds)
print("Test Accuracy: ", (preds == y_test).sum()/len(y_test))
print("Test Macro F1-Score: ", F1)
print("Test Precision: ", precision)
print("Test Recall: ", recall)

y_train = y_train.reshape(-1)
preds = best_xgb_model.predict(X_train)
F1 = f1_score(y_train, preds, average='macro')
precision = precision_score(y_train, preds)
recall = recall_score(y_train, preds)
print("Train Accuracy: ", (preds == y_train).sum()/len(y_train))
print("Train Macro F1-Score: ", F1)
print("Train Precision: ", precision)
print("Train Recall: ", recall)

In [ ]:
#params = {'objective': 'binary:logistic', 'base_score': [0.0032416375],
          #'booster': 'gbtree', 'callbacks': None, 'colsample_bylevel': None,
          #'colsample_bynode': None, 'colsample_bytree': None, 'device': None,
          #'early_stopping_rounds': None, 'enable_categorical': False,
          #'eval_metric': None, 'feature_types': None, 'feature_weights': None,
          #'gamma': None, 'grow_policy': None, 'importance_type': None,
          #'interaction_constraints': None, 'learning_rate': None,
          #'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None,
          #'max_delta_step': None, 'max_depth': None, 'max_leaves': None,
          #'min_child_weight': None, 'monotone_constraints': None,
          #'multi_strategy': None, 'n_estimators': 2000, 'n_jobs': None,
          #'num_parallel_tree': None, 'random_state': None, 'reg_alpha': None,
          #'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': scale_pos_weight_value,
          #subsample': None, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}
#model = XGBClassifier(**params)
#model.fit(X_train, y_train)

In [ ]:
df = pd.read_csv('final_test.csv')
X_df = df[labels]
X = X_df.to_numpy()

In [ ]:
# Make predictions
predictions = final_xgb.predict(X)
#predictions = model.predict(X)

# Create submission file (format: TxnID,FraudLabel)
submission_df = pd.DataFrame({
    'TxnID': df['TxnID'],
    'FraudLabel': predictions
})

# Save predictions
submission_df.to_csv("retailbanking_challenge1_predictions.csv", index=False)
print(len(X))
print(f"✅ Predictions saved: {submission_df.shape[0]} predictions")
print(f"   Preview: {submission_df.head(3)}")
print(f"   Fraud rate: {predictions.mean():.3f} ({predictions.sum()} fraud cases out of {len(predictions)})")

In [ ]:
from agentds import BenchmarkClient

client = BenchmarkClient(
    api_key="adsb_hdm1DRk1iW2I1VA84Oc9jz6z_1756090832",        # Get from your team dashboard
    team_name="agi"     # Your exact team name
)

# 3. Submit Predictions

# Submit predictions to the competition
print("🚀 Submitting predictions...")

try:
    result = client.submit_prediction("Retailbanking", 1, "retailbanking_challenge1_predictions.csv")
    
    if result['success']:
        print("✅ Submission successful!")
        print(f"   📊 Score: {result['score']:.4f}")
        print(f"   📏 Metric: {result['metric_name']}")
        print(f"   ✔️  Validation: {'Passed' if result['validation_passed'] else 'Failed'}")
    else:
        print("❌ Submission failed!")
        print(f"   Error details: {result.get('details', {}).get('validation_errors', 'Unknown error')}")
        
except Exception as e:
    print(f"💥 Submission error: {e}")
    print("🔧 Check your API key and team name are correct!")

print("\n🎯 Next steps:")
print("   1. Try incorporating relevant information outside this table!")
print("   2. Move on to Retail Banking Challenge 2!")
